In [29]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import itertools
from itertools import product



options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

In [31]:
# Parameters
num_objs = 100  # Define the size of the problem
capacity = 30  # Define knapsack capacity

# Randomly generate weights and values
np.random.seed(0)  # Seed for reproducibility
weights = np.random.randint(1, 5, num_objs)  
values = np.random.randint(0, 10, num_objs) 

# Create a symmetric matrix for the quadratic term
Q = np.random.rand(num_objs, num_objs) 
Q = (Q + Q.T) / 2  # Make it symmetric

# Set up the environment
options = {"OutputFlag": 1}  # OutputFlag to display output in console

# Initialize Gurobi environment and model
with gp.Env(params=options) as env:
    with gp.Model(env=env) as model:
        
        # Define binary variables for knapsack items
        x = model.addVars(num_objs, vtype=  GRB.BINARY, name="x")
        
        # Set the objective function with quadratic terms
        obj = gp.quicksum(values[i] * x[i] for i in range(num_objs))
        obj += gp.quicksum(Q[i, j] * x[i] * x[j] for i in range(num_objs) for j in range(i+1, num_objs))
        model.setObjective(obj, GRB.MAXIMIZE)
        
        # Add the capacity constraint
        model.addConstr(gp.quicksum(weights[i] * x[i] for i in range(num_objs)) <= capacity, "Capacity")
        
        # Optimize the model
        model.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B83)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 100 columns and 100 nonzeros
Model fingerprint: 0xeba12bf7
Model has 4950 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 9e+00]
  QObjective range [3e-02, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve time: 0.01s
Presolved: 1 rows, 100 columns, 100 nonzeros
Presolved model has 5050 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)
Found heuristic solution: objective 71.4830336

Root relaxation: objective 5.069308e+02, 74 iterations, 0.00 seconds (0.00 work units)

    Nodes    |

# Sparse quadratic term

In [42]:
# Parameters
num_objs = 500  # Define the size of the problem
capacity = 30  # Define knapsack capacity

# Randomly generate weights and values
np.random.seed(0)  # Seed for reproducibility
weights = np.random.randint(1, 5, num_objs)  
values = np.random.randint(0, 10, num_objs) 

# Create a symmetric matrix for the quadratic term
density = 0.01
upper_triangular = np.triu((np.random.rand(num_objs, num_objs) < density).astype(int), k=1)
Q = upper_triangular + upper_triangular.T
np.fill_diagonal(Q, 0)

# Set up the environment
options = {"OutputFlag": 1}  # OutputFlag to display output in console

# Initialize Gurobi environment and model
with gp.Env(params=options) as env:
    with gp.Model(env=env) as model:
        
        # Define binary variables for knapsack items
        x = model.addVars(num_objs, vtype=  GRB.BINARY, name="x")
        
        # Set the objective function with quadratic terms
        obj = gp.quicksum(values[i] * x[i] for i in range(num_objs))
        obj += gp.quicksum(Q[i, j] * x[i] * x[j] for i in range(num_objs) for j in range(i+1, num_objs))
        model.setObjective(obj, GRB.MAXIMIZE)
        
        # Add the capacity constraint
        model.addConstr(gp.quicksum(weights[i] * x[i] for i in range(num_objs)) <= capacity, "Capacity")
        
        # Optimize the model
        model.optimize()
        result = np.array(model.x)

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B83)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information